# Machine Learing Development for Sales Forecasting

Another possible solution is to use machine learning algorithms to create predictive models based on sales data, to identify the market characteristics that most influence the the amount of sales. Once we identify these characteristics, they can focus their monitoring efforts on product with a higher likelihood of improving sales. 

The case of big mart sales can be treated as a regression problem. The goal is to forecast sales. The target variable in this case can be specified as a nominal variable, which are sales amount in a year

A wide range of machine learning algorithms can be used to solve the problem, we will use:

+ Linear regression techniques, such as LinearRegression, which use a linear function to predict the amount of sales.
+ Decision tree algorithms can be used to simulate more complex interactions between input factors and the target variable. RandomForestRegressor is a machine-learning model built from a collection of decision trees, each of which is trained on a different subset of training data. The program averages the predictions of all the trees to provide a final prediction.

To evaluate the performance of machine learning models, we will use various metrics, such as `mean absolute error (MAE)`, `mean squared error (MSE)`, `root mean squared error (RMSE)`, `R-Squared`, `mean absolute percentage error (MAPE)`.



In [2]:
#@title importing necesary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pingouin as pg
import pickle
#import scikitplot as skplt

from typing import List, Tuple

from pandas.api.types import CategoricalDtype
from statsmodels.stats.contingency_tables import Table2x2
from scipy.stats import randint

from sklearn.compose import make_column_selector,make_column_transformer, ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer, OrdinalEncoder,  OneHotEncoder

from sklearn.experimental import enable_halving_search_cv 
from sklearn.metrics import accuracy_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
                                                        
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score,\
                                    learning_curve, validation_curve, RandomizedSearchCV, HalvingRandomSearchCV,\
                                    KFold
                                    
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor, ExtraTreesRegressor

import warnings
warnings.filterwarnings('ignore')

C:\Users\Administrator\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


We have performed some feature engineering in the data exploration phase and some feature have therefore become obsolete. We will have to drop those columns. Also we will be using different encoders for the category columns based on its characteristics.

In [3]:
# load the dataset
# load the data (cleaned)
bigmart = pd.read_csv('bigmart_cleaned.csv')

bigmart_copy = bigmart.copy() # create a copy of the data
bigmart_copy.head()

,Item_Weight,Item_Fat_Content,Item_Visibility (%),Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Age,Item_Type_Category,Item_MRP_Category
0,9.30,NaN,200.0,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380,14,Food,Very High
1,5.92,Regular,200.0,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228,4,Drink,Low
2,17.50,NaN,200.0,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700,14,Food,High
3,19.20,Regular,1200.0,Fruits and Vegetables,182.0950,OUT010,Small,Tier 3,Grocery Store,732.3800,15,Food,High
4,8.93,No Fat,600.0,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052,26,Non-Consumable,Low


In [4]:
# @let start with data preprocessing

# set the seed
seed = 200

# set the feature and target variable
X = bigmart_copy.drop('Item_Outlet_Sales', axis = 1)
y = bigmart_copy.Item_Outlet_Sales

# split data into train and test set
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle = True, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

# create a list of numerical column
num_selector = make_column_selector(dtype_exclude='object')

# create two set of of category selector, one of ordinal type and 
# other of nominal type
cat_selector_nom = make_column_selector(dtype_include='object')
cat_selector_ord= make_column_selector(dtype_include= 'category')

# sekect this column from the data
num_cols = num_selector(X)
cat_cols_ord = cat_selector_ord(X)
cat_cols_nom = cat_selector_nom(X)

# initiate the preprocessor for each selctor
num_preprocessor = RobustScaler()
cat_selector_nom_preprocessor = OneHotEncoder()
cat_selector_ord_preprocessor = OrdinalEncoder()

# set the preprocessor
preprocesor = ColumnTransformer([
    ('RobustScaler', num_preprocessor, num_cols),
    ('OneHotEncoder', cat_selector_nom_preprocessor, cat_cols_nom),
    ('OrdinalEncoder', cat_selector_ord_preprocessor, cat_cols_ord)
])

# create a machine model pipeline
pipelines = {
    'Linear Regression':make_pipeline(preprocesor, LinearRegression()),
    'Random Forest Regressor': make_pipeline(preprocesor, RandomForestRegressor()),
    'Gradient Boost Regression':make_pipeline(preprocesor, GradientBoostingRegressor()),
    'Extra Tree Regressor':make_pipeline(preprocesor, ExtraTreesRegressor())
}


In [6]:
# create a function to get model metrics
def metrics_from_pipes(pipes_dict):
    ''' 
    This function takes as input a dictionary of ML pipelines  and
    returns a table all the train and test metrics for
    each model in the dictionary
    '''
        
    # create a list for each evaluation metrics to store the metric
    # for each model
    train_mae = []
    train_mape = []
    train_rsquared = []
    train_rmse = []
    
    val_mae = []
    val_mape = []
    val_rsquared = []
    val_rmse = []
    
    # fit model and get the metrics
    for name, pipeline in pipes_dict.items():
        
        pipeline.fit(X_train, y_train)
        y_pred_val = pipeline.predict(X_val)
        y_pred_train = pipeline.predict(X_train)
        

        # get the evaluation Metrics for train
        train_mae.append(mean_absolute_error(y_train, y_pred_train))
        train_mape.append(mean_absolute_percentage_error(y_train, y_pred_train))
        train_rmse.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
        train_rsquared.append(r2_score(y_train, y_pred_train))
        
        # get the evaluation metrics for validation
        val_mae.append(mean_absolute_error(y_val, y_pred_val))
        val_mape.append(mean_absolute_percentage_error(y_val, y_pred_val))
        val_rmse.append(np.sqrt(mean_squared_error(y_val, y_pred_val)))
        val_rsquared.append(r2_score(y_val, y_pred_val))
        
    # aggregate the performance metric lists into seperate dataframes
    train_metrics = {
            'Model':list(pipes_dict.keys()),
            'MAE':train_mae,
            'MAPE':train_mape,
            'RMSE':train_rmse,
            'RSquared':train_rsquared
        }
        
    train_metrics_data = pd.DataFrame.from_dict(train_metrics)
    #columns = ['model', 'MAE', 'MAPE', 'RMSE', 'RSquared']
    val_metrics = {
            'Model':list(pipes_dict.keys()),
            'MAE':val_mae,
            'MAPE':val_mape,
            'RMSE':val_rmse,
            'RSquared':val_rsquared            
        }
        
    val_metrics_data = pd.DataFrame(val_metrics)
        
    #Merging metrics from train and test set
    train_val_metrics = train_metrics_data.merge(val_metrics_data,
                                               on = 'Model',
                                               how = 'left',
                                               suffixes = ('_train', '_val'))
        
    # sorting columns 
    train_val_metrics = train_val_metrics.reindex(columns = ['Model',
                                                               'MAE_train',
                                                                'MAPE_train',
                                                                'RMSE_train',
                                                                'RSquared_train',
                                                                'MAE_val',
                                                                'MAPE_val',
                                                                'RMSE_val',
                                                                'RSquared_val'])
        
    
    
    return train_val_metrics.set_index('Model').transpose()
    #return train_metrics
# get the metrics table
metrics_table = metrics_from_pipes(pipelines)
#print('Table 1: Base Models Metrics')
#metrics_table.style.background_gradient(cmap = Blues)
metrics_table

Model,Linear Regression,Random Forest Regressor,Gradient Boost Regression,Extra Tree Regressor
MAE_train,829.102341,287.245080,712.963727,2.483015e-12
MAPE_train,1.030237,0.210412,0.538245,1.147776e-15
RMSE_train,1120.554271,413.311365,1012.399202,3.676770e-12
RSquared_train,0.559829,0.940116,0.640698,1.000000e+00
MAE_val,855.925441,798.070178,773.454690,8.455523e+02
MAPE_val,1.039549,0.562227,0.544038,5.854934e-01
RMSE_val,1148.654990,1151.876256,1112.580980,1.229804e+03
RSquared_val,0.587671,0.585355,0.613163,5.273533e-01
